In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
df_fraud_all = pd.read_csv("data/Fraud_Detection_sql.csv")

Not labeled!


fraud_all 제대로 불러오기 했는지 검사

In [5]:
df_fraud_all.head(2)

,TRANS_YEAR,IS_COVID_YEAR,TRANS_MONTH,TRANS_DAY,TRANS_DAY_SIMPLIFIED,TRANS_HOUR,TRANS_HOUR_SIMPLIFIED,CATEGORY,AMT,GENDER,CITY,CITY_POP,STATE,JOB,AGE,IS_FRAUD,CC_NUM,log_AMT,BC_CITY_POP
0,2019,False,1,1,0,0,0,0,4.97,False,Moravian Falls,3495,0,"Psychologist, counselling",35,False,2703186189652095,1.603420,5.391860
1,2019,False,1,1,0,0,0,1,107.23,False,Orient,149,1,Special educational needs teacher,45,False,630423337322,4.674976,3.851125


In [6]:
df_fraud_all["IS_FRAUD"].value_counts()

False    1842743
True        9651
Name: IS_FRAUD, dtype: int64

In [7]:
not_fraud_count = df_fraud_all["IS_FRAUD"].value_counts()[0]
fraud_count = df_fraud_all["IS_FRAUD"].value_counts()[1]
fraud_ratio = fraud_count / (not_fraud_count + fraud_count)
print("fraud의 비율은 "+str(fraud_ratio)+"(%f%%)입니다." %(fraud_ratio*100))

fraud의 비율은 0.005210014716091717(0.521001%)입니다.


0.5%이니 995:5 비율로 데이터가 존재한다.
비율을 0.2로 맞추고 싶으면 20:5로 줄인다.
따라서 not_fraud데이터 갯수에서 20/995가 내가 실제로 3개의 데이터 셋으로 나눌 대상 크기가 된다.

In [8]:
df_not_fraud=df_fraud_all[df_fraud_all["IS_FRAUD"] == 0]

df_not_fraud_reduced = df_not_fraud.sample(frac = 20/995)

df_fraud=df_fraud_all[df_fraud_all["IS_FRAUD"] == 1]

df_fraud_all_reduced = pd.concat([df_not_fraud_reduced, df_fraud])

#비율을 맞춘 fraud 데이터프레임의 갯수 확인
df_fraud_all_reduced["IS_FRAUD"].value_counts()

False    37040
True      9651
Name: IS_FRAUD, dtype: int64

In [9]:
not_fraud_count = df_fraud_all_reduced["IS_FRAUD"].value_counts()[0]
fraud_count = df_fraud_all_reduced["IS_FRAUD"].value_counts()[1]
fraud_ratio = fraud_count / (not_fraud_count + fraud_count)
print("fraud의 비율은 "+str(fraud_ratio)+"(%f%%)입니다." %(fraud_ratio*100))

#df_fraud_all_reduced 데이터프레임을 사용하도록 한다.

fraud의 비율은 0.20669936390310767(20.669936%)입니다.


이제 DF를 train, validation, test 로 나눈다. 보통 6:2:2로 나눈다.

In [10]:
from sklearn.model_selection import train_test_split

df_fraud_all_reduced_y = df_fraud_all_reduced["IS_FRAUD"]
df_fraud_all_reduced_x = df_fraud_all_reduced.drop(columns=["IS_FRAUD"], inplace=False)

print(df_fraud_all_reduced_x.shape)
print(df_fraud_all_reduced_y.shape)

df_train2_x, df_test_x, df_train2_y, df_test_y = train_test_split(df_fraud_all_reduced_x, df_fraud_all_reduced_y, train_size = 0.8, random_state = 42, shuffle = True, stratify=df_fraud_all_reduced_y)
df_train_x, df_val_x, df_train_y, df_val_y = train_test_split(df_train2_x, df_train2_y, train_size = 0.75, random_state = 42, shuffle = True, stratify=df_train2_y)

print(df_train_x.shape)
print(df_train_y.shape)
print(df_val_x.shape)
print(df_val_y.shape)
print(df_test_x.shape)
print(df_test_y.shape)

os.mkdir("data/splitted")
df_train_x.to_csv("data/splitted/Fraud_Detection_train_features.csv", index=False)
df_train_y.to_csv("data/splitted/Fraud_Detection_train_target.csv", index=False)
df_val_x.to_csv("data/splitted/Fraud_Detection_validation_features.csv", index=False)
df_val_y.to_csv("data/splitted/Fraud_Detection_validation_target.csv", index=False)
df_test_x.to_csv("data/splitted/Fraud_Detection_test_features.csv", index=False)
df_test_y.to_csv("data/splitted/Fraud_Detection_test_target.csv", index=False)

#df_train_x -> 훈련 데이터의 feature테이블
#df_train_y -> 훈련 데이터의 target배열
#df_val_x -> 검증 데이터의 feature테이블
#df_val_y -> 검증 데이터의 target배열
#df_test_x -> 시험 데이터의 feature테이블
#df_test_y -> 시험 데이터의 target배열

(46691, 18)
(46691,)
(28014, 18)
(28014,)
(9338, 18)
(9338,)
(9339, 18)
(9339,)
